In [25]:
# server_ip = "cloudflare-quic.com/"
nginx = {
    "name": "nginx",
    "ip": "3.71.47.197",
}
h2o = {
    "name": "h2o",
    "ip": "35.156.45.133",
}

iperf_port = "6969"

servers = (nginx, h2o)

## Bandwidth and latency test

### Latency

In [32]:
for server in servers:
    print("")
    print(f"==================== PING {server['name']} ==========================")
    !ping -c 5 {server["ip"]}


==================== PING nginx ==========================
PING 3.71.47.197 (3.71.47.197) 56(84) bytes of data.
64 bytes from 3.71.47.197: icmp_seq=1 ttl=63 time=0.587 ms
64 bytes from 3.71.47.197: icmp_seq=2 ttl=63 time=0.513 ms
64 bytes from 3.71.47.197: icmp_seq=3 ttl=63 time=0.566 ms
64 bytes from 3.71.47.197: icmp_seq=4 ttl=63 time=0.552 ms
64 bytes from 3.71.47.197: icmp_seq=5 ttl=63 time=0.541 ms

--- 3.71.47.197 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4103ms
rtt min/avg/max/mdev = 0.513/0.551/0.587/0.024 ms

==================== PING h2o ==========================
PING 35.156.45.133 (35.156.45.133) 56(84) bytes of data.
64 bytes from 35.156.45.133: icmp_seq=1 ttl=63 time=0.552 ms
64 bytes from 35.156.45.133: icmp_seq=2 ttl=63 time=0.528 ms
64 bytes from 35.156.45.133: icmp_seq=3 ttl=63 time=0.523 ms
64 bytes from 35.156.45.133: icmp_seq=4 ttl=63 time=0.565 ms
64 bytes from 35.156.45.133: icmp_seq=5 ttl=63 time=0.543 ms

--- 35.156.45.133 pin

### Bandwidth - TCP & UDP

In [28]:
# TODO: Make sure iperf is running on all servers.
!iperf -c {nginx["ip"]} -p {iperf_port}
!iperf -u -c {server_ip} -p {iperf_port} -b 0

Connecting to host 3.71.47.197, port 6969
[  5] local 172.31.10.40 port 58980 connected to 3.71.47.197 port 6969
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec   457 MBytes  3.84 Gbits/sec  291    325 KBytes       
[  5]   1.00-2.00   sec   500 MBytes  4.19 Gbits/sec  259    576 KBytes       
[  5]   2.00-3.00   sec   470 MBytes  3.94 Gbits/sec  156    605 KBytes       
[  5]   3.00-4.00   sec   495 MBytes  4.15 Gbits/sec  206    428 KBytes       
[  5]   4.00-5.00   sec   464 MBytes  3.89 Gbits/sec  207    482 KBytes       
[  5]   5.00-6.00   sec   472 MBytes  3.96 Gbits/sec  209    560 KBytes       
[  5]   6.00-7.00   sec   471 MBytes  3.96 Gbits/sec  169    457 KBytes       
[  5]   7.00-8.00   sec   404 MBytes  3.39 Gbits/sec  200    475 KBytes       
[  5]   8.00-9.00   sec   319 MBytes  2.67 Gbits/sec  154    174 KBytes       
[  5]   9.00-10.00  sec   391 MBytes  3.28 Gbits/sec  115    441 KBytes       
- - - - - - - - - - - - - - - - 

### Measuring QUIC performance

#### h2 load output parsing

In [53]:
def get_throughput(res):
    fin_line = [l for l in res if l.startswith("finished in ")][0]
    return float(fin_line.split(" ")[-1][:-4])

def get_rps(res):
    fin_line = [l for l in res if l.startswith("finished in ")][0]
    return float(fin_line.split(" ")[3])

def get_success_pct(res):
    req_line = [l for l in res if l.startswith("requests:")][0]
    items = req_line.split(" ")
    return float(items[7])/float(items[1])

#### Bulk Throughput

Get throughput measurment from the client.

In [47]:
for server in servers:
    for protocol in ("h2", "h3",):
        print(f"bulk throughput test {protocol} - {server['name']}")
        # Request the file first, just to warm up caches.
        _ = !h2load -n1 -c1 --npn-list={protocol} https://{server['ip']}/2g.txt
        for i in range(2):
            res = !h2load -n1 -c1 --npn-list={protocol} https://{server['ip']}/2g.txt
            print(f"{get_throughput(res)} MB/s (succ: {get_success_pct(res)})")

bulk throughput test h2 - nginx
428.95 MB/s (succ: 1.0)
551.17 MB/s (succ: 1.0)
bulk throughput test h3 - nginx
90.43 MB/s (succ: 1.0)
92.12 MB/s (succ: 1.0)
bulk throughput test h2 - h2o
451.74 MB/s (succ: 1.0)
546.93 MB/s (succ: 1.0)
bulk throughput test h3 - h2o
255.09 MB/s (succ: 1.0)
247.68 MB/s (succ: 1.0)


In [78]:
for server in servers:
    for protocol in ("h2", "h3"):
        print(f"rps test {protocol} - {server['name']}")
        for i in range(2):
            res = !h2load -n1000000 -t 2 -c 50 -m 100 --npn-list={protocol} https://{server['ip']}/index.html
            print(f"{get_rps(res)} req/s (succ: {get_success_pct(res):.3f})")

rps test h2 - nginx
41148.74 req/s (succ: 0.050)
44307.7 req/s (succ: 0.050)
rps test h3 - nginx
71903.44 req/s (succ: 0.050)
70238.63 req/s (succ: 0.050)
rps test h2 - h2o
246581.82 req/s (succ: 1.000)
239029.16 req/s (succ: 1.000)
rps test h3 - h2o
162569.96 req/s (succ: 1.000)
160809.53 req/s (succ: 1.000)


In [75]:
res[-20:]

['ngtcp2_conn_read_pkt: ERR_DRAINING',
 'Warm-up phase is over for thread #1.',
 'Main benchmark duration is started for thread #1.',
 'Warm-up phase is over for thread #0.',
 'Main benchmark duration is started for thread #0.',
 'Main benchmark duration is over for thread #1. Stopping all clients.',
 'Stopped all clients for thread #1',
 'Main benchmark duration is over for thread #0. Stopping all clients.',
 'Stopped all clients for thread #0',
 '',
 'finished in 8.01s, 0.00 req/s, 0B/s',
 'requests: 0 total, 0 started, 0 done, 0 succeeded, 0 failed, 0 errored, 0 timeout',
 'status codes: 0 2xx, 0 3xx, 0 4xx, 0 5xx',
 'traffic: 0B (0) total, 5.20MB (5450000) headers (space savings 37.71%), 1.10MB (1150000) data',
 'UDP datagram: 1278 sent, 7470 received',
 '                     min         max         mean         sd        +/- sd',
 'time for request:        0us         0us         0us         0us     0.00%',
 'time for connect:        0us         0us         0us         0us     0.0